In [8]:
import pandas as pd
import yfinance as yf
import datetime
from datetime import date, timedelta
import plotly.graph_objects as go
import plotly.express as px
today = date.today()
%pip install nbformat
%pip install praw
%pip install textblob
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
from textblob import TextBlob
import praw


  Using cached nbformat-5.10.4-py3-none-any.whl.metadata (3.6 kB)
  Using cached fastjsonschema-2.21.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached jupyter_core-5.7.2-py3-none-any.whl.metadata (3.4 kB)
  Using cached traitlets-5.14.3-py3-none-any.whl.metadata (10 kB)
  Using cached attrs-25.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached jsonschema_specifications-2024.10.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
  Using cached rpds_py-0.22.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (4.2 kB)
Using cached nbformat-5.10.4-py3-none-any.whl (78 kB)
Using cached fastjsonschema-2.21.1-py3-none-any.whl (23 kB)
Using cached jsonschema-4.23.0-py3-none-any.whl (88 kB)
Using cached jupyter_core-5.7.2-py3-none-any.whl (28 kB)
Using cached traitlets-5.14.3-py3-none-any.whl (85 kB)
Using cached attrs-25.1.0-py3-none-any.whl (63 kB)
Using cached jsonschema_spe

In [9]:

d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=365*3) #going back 1 year
d2 = d2.strftime("%Y-%m-%d") #date format for the api
start_date = d2 #start date for the api

ticker='NVDA'
# Collecting the stock price data of H&M using the yfinance API of Yahoo Finance.
data = yf.download(ticker, start=start_date,
                              end=end_date,
                              progress=False) 

# Sorting the data in descending order of date
data_unsorted=data.copy()
data = data.sort_index(ascending=False)

# Printing the sorted data
print(data)
print(data_unsorted)

print(data.columns)


Price            Close        High         Low        Open     Volume
Ticker            NVDA        NVDA        NVDA        NVDA       NVDA
Date                                                                 
2025-01-27  118.419998  128.399994  116.699997  124.800003  818830900
2025-01-24  142.619995  148.970001  141.880005  148.369995  234657600
2025-01-23  147.220001  147.229996  143.720001  145.050003  155915500
2025-01-22  147.070007  147.789993  143.669998  144.660004  237651400
2025-01-21  140.830002  141.830002  137.089996  139.160004  197749000
...                ...         ...         ...         ...        ...
2022-02-04   24.278622   24.594097   23.592762   23.932198  354829000
2022-02-03   23.908236   25.035361   23.740515   24.417389  410178000
2022-02-02   25.200089   25.774133   24.512231   25.751172  543419000
2022-02-01   24.597094   25.103252   23.850334   25.062319  518925000
2022-01-31   24.445341   24.468304   23.013722   23.143506  564680000

[750 rows x 5 colum

In [10]:
# Install nbformat to fix the ValueError

#data2 = data.copy()
# Flatten the multi-index columns
data.columns = data.columns.get_level_values(0)

# "Date" is index in this dataset. Below is how to convert this index into column.
data["Date"] = data.index
data = data[["Date", "Open", "High", "Low", "Close", "Volume"]]
data.reset_index(drop=True, inplace=True)
print(data.head())
print(data.tail())

data.to_csv(f"{ticker}.csv", index=False)
# Convert the "Date" column to datetime format
data["Date"] = pd.to_datetime(data["Date"])

#do the same for the unsorted data
data_unsorted.columns = data_unsorted.columns.get_level_values(0)
data_unsorted["Date"] = data_unsorted.index
data_unsorted = data_unsorted[["Date", "Open", "High", "Low", "Close", "Volume"]]
data_unsorted.reset_index(drop=True, inplace=True)
data_unsorted["Date"] = pd.to_datetime(data_unsorted["Date"])




Price       Date        Open        High         Low       Close     Volume
0     2025-01-27  124.800003  128.399994  116.699997  118.419998  818830900
1     2025-01-24  148.369995  148.970001  141.880005  142.619995  234657600
2     2025-01-23  145.050003  147.229996  143.720001  147.220001  155915500
3     2025-01-22  144.660004  147.789993  143.669998  147.070007  237651400
4     2025-01-21  139.160004  141.830002  137.089996  140.830002  197749000
Price       Date       Open       High        Low      Close     Volume
745   2022-02-04  23.932198  24.594097  23.592762  24.278622  354829000
746   2022-02-03  24.417389  25.035361  23.740515  23.908236  410178000
747   2022-02-02  25.751172  25.774133  24.512231  25.200089  543419000
748   2022-02-01  25.062319  25.103252  23.850334  24.597094  518925000
749   2022-01-31  23.143506  24.468304  23.013722  24.445341  564680000


In [11]:
import plotly.graph_objects as go
import plotly.express as px

# Bar plot with correct date formatting
figure = go.Figure(data=[go.Bar(x=data["Date"], y=data["Open"])])

figure.update_layout(
    title=f"Open Prices of {ticker} from {start_date} to {end_date} with Bar Plot",
    xaxis=dict(
        tickformat="%b %d, %Y",
        tickmode='array',
        tickvals=data["Date"][::len(data)//10],
        ticktext=[d.strftime('%b %d, %Y') for d in data["Date"][::len(data)//10]]
    )
)
figure.show()

# Line plot with time period selectors and correct date formatting
figure = px.line(data, x='Date', y='Close', title=f"Close Prices of {ticker} from {start_date} to {end_date} with Line Plot")
figure.update_xaxes(
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=3, label="3m", step="month", stepmode="backward"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    ),
    tickformat="%b %d, %Y",
    tickmode='array',
    tickvals=data["Date"][::len(data)//10],
    ticktext=[d.strftime('%b %d, %Y') for d in data["Date"][::len(data)//10]]
)
figure.show()

# Candlestick chart with correct date formatting
figure = go.Figure(data=[go.Candlestick(
    x=data["Date"], 
    open=data["Open"], 
    high=data["High"], 
    low=data["Low"], 
    close=data["Close"],
    hovertext='Date: %{x}<br>Open: %{open}<br>High: %{high}<br>Low: %{low}<br>Close: %{close}<extra></extra>'
)])
figure.update_layout(
    title=f'Stock Price Analysis of {ticker}',
    xaxis_rangeslider_visible=False,
    xaxis=dict(
        tickformat="%b %d, %Y",
        tickmode='array',
        tickvals=data["Date"][::len(data)//10],
        ticktext=[d.strftime('%b %d, %Y') for d in data["Date"][::len(data)//10]]
    )
)
figure.show()

# Line plot with time period selectors and correct date formatting
figure = px.line(data, x='Date', y='Close', title=f'Stock Market Analysis with Time Period Selectors of {ticker}')
figure.update_xaxes(
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=3, label="3m", step="month", stepmode="backward"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    ),
    tickformat="%b %d, %Y",
    tickmode='array',
    tickvals=data["Date"][::len(data)//10],
    ticktext=[d.strftime('%b %d, %Y') for d in data["Date"][::len(data)//10]]
)
figure.show()

# Line plot with candlestick overlay and time period selectors
figure = go.Figure()

# Add candlestick data
figure.add_trace(go.Candlestick(
    x=data["Date"],
    open=data["Open"],
    high=data["High"],
    low=data["Low"],
    close=data["Close"],
    name='Candlestick',
    hovertext='Date: %{x}<br>Open: %{Open}<br>High: %{High}<br>Low: %{Low}<br>Close: %{Close}<extra></extra>'
))

# Add line plot data
figure.add_trace(go.Scatter(
    x=data["Date"],
    y=data["Close"],
    mode='lines',
    name='Close Price'
))

# Update layout with time period selectors and correct date formatting
figure.update_layout(
    title=f'Candlestick analysis with Line Plot of {ticker}',

    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=3, label="3m", step="month", stepmode="backward"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        tickformat="%b %d, %Y",
        tickmode='array',
        tickvals=data["Date"][::len(data)//10],
        ticktext=[d.strftime('%b %d, %Y') for d in data["Date"][::len(data)//10]]
    )
)

figure.show()


In [12]:
##predixcts the stock price for the next 30 days but uses the data and predictts the date before

## Build The Model with LSTM and RMSProp


In [13]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
import pandas as pd
from sklearn.metrics import mean_squared_error

class StockPricePredictor:
    def __init__(self, data, time_step=60, train_split=0.7): 
        self.data = data  # Stock price data
        self.time_step = time_step  # Lookback period
        self.train_split = train_split  # Train/test split ratio
        self.scaler = MinMaxScaler(feature_range=(0, 1))  # Data normalization
        self.model = None  # LSTM model
        self.data_normalized = self.normalize_data()  # Normalize the data
        self.split_data()  # Split data into train/test
        self.create_datasets()  # Create training and testing datasets

    def normalize_data(self):
        numeric_data = self.data[['Open', 'High', 'Low', 'Close', 'Volume']]
        normalized = self.scaler.fit_transform(numeric_data)
        self.data['Normalized_Open'] = normalized[:, 0]
        self.data['Normalized_High'] = normalized[:, 1]
        self.data['Normalized_Low'] = normalized[:, 2]
        self.data['Normalized_Close'] = normalized[:, 3]
        self.data['Normalized_Volume'] = normalized[:, 4]
        return normalized

    def split_data(self):
        self.train_data = self.data_normalized  
        self.test_data = self.data_normalized  

    def create_dataset(self, data):
        X, Y = [], []
        for i in range(len(data) - self.time_step):
            if i + self.time_step < len(data):
                X.append(data[i:(i + self.time_step), :])
                Y.append(data[i + self.time_step, 3])  
        return np.array(X), np.array(Y)

    def create_datasets(self):
        self.X_train, self.Y_train = self.create_dataset(self.train_data)
        self.X_test, self.Y_test = self.create_dataset(self.test_data)

    def build_model(self):
        self.model = tf.keras.models.Sequential([
            tf.keras.layers.LSTM(units=50, return_sequences=True, input_shape=(self.time_step, self.X_train.shape[2])),
            tf.keras.layers.Dropout(0.1),
            tf.keras.layers.LSTM(units=50),
            tf.keras.layers.Dense(1)
        ])
        self.model.compile(loss='mean_squared_error', optimizer='Adamax', metrics=['mean_absolute_error'])
        self.model.summary()

    def train_model(self, epochs=100, batch_size=64):
        # EarlyStopping with restore_best_weights set to True
        self.model.fit(self.X_train, self.Y_train, epochs=epochs, 
                    validation_data=(self.X_test, self.Y_test), 
                    verbose=1, batch_size=batch_size, shuffle=False,
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                            patience=10,  # Feel free to adjust this value
                                                            restore_best_weights=True)])

    def predict(self):
        # Predict the test data
        test_predicted = self.model.predict(self.X_test)
        # Evaluate loss and MAE
        test_loss, test_mae = self.model.evaluate(self.X_test, self.Y_test, verbose=0)
        # Calculate RMSE using sklearn's mean_squared_error
        test_rmse = np.sqrt(mean_squared_error(self.Y_test, test_predicted))
        # Output all metrics
        print(f"Testing Mean Absolute Error: {test_mae:.4f}")
        print(f"Testing Loss (MSE): {test_loss:.4f}")
        print(f"Testing Root Mean Squared Error (RMSE): {test_rmse:.4f}")  # RMSE displayed here
        print(f"Predicted Close price for the next {len(test_predicted)} days:")
        
        return test_predicted


    def inverse_transform(self, predicted, target_column=2):
        predicted_full = np.zeros((predicted.shape[0], self.scaler.min_.shape[0]))
        predicted_full[:, target_column] = predicted.flatten()
        inversed = self.scaler.inverse_transform(predicted_full)
        return inversed[:, target_column]

    def predict_next_day(self):
        recent_data = self.data_normalized[-self.time_step:, :]
        last_sequence = recent_data.reshape(1, self.time_step, self.X_train.shape[2])
        next_day_close = self.model.predict(last_sequence)
        last_date = pd.to_datetime(self.data.iloc[-1]['Date'])
        next_day = last_date + pd.DateOffset(days=1)
        while next_day.weekday() >= 5:  # Skip weekends
            next_day += pd.DateOffset(days=1)

        # Return predicted close price for the next day
        return next_day, self.inverse_transform(next_day_close)[0]

    def plot_predictions(self, actual, predicted):
        figure = go.Figure()
        figure.add_trace(go.Scatter(x=actual.index, y=actual['High'], mode='lines', name='Actual High Price'))
        figure.add_trace(go.Scatter(x=actual.index, y=predicted, mode='lines', name='Predicted High Price'))
        figure.update_layout(title='Actual vs Predicted High Price')
        figure.show()

    def run(self):
        self.build_model()
        self.train_model()
        test_loss, test_mae = self.model.evaluate(self.X_test, self.Y_test, verbose=0)
        print(f"Test Loss (MSE): {test_loss:.4f}")
        print(f"Test Mean Absolute Error (MAE) for Close price: {test_mae:.4f}")  # MAE for Close price
        print(f"Test Root Mean Squared Error (RMSE) for Close price: {np.sqrt(test_mae):.4f}")
        print("Predictions for the test data:")
        test_predicted = self.predict()
        next_day, next_day_close = self.predict_next_day()  # Use the predicted close price here
        print(f"Data ended as of the date: {self.data.iloc[-1]['Date'].date()}")
        print(f"Today's Close price for {ticker}: {self.data.iloc[-1]['Close']:.2f}")
        print(f"Predicted next weekday Close price of {ticker} for {next_day.date()}: {next_day_close:.2f}")
        self.plot_predictions(self.data, self.inverse_transform(test_predicted))
        return next_day_close

predictor = StockPricePredictor(data_unsorted, time_step=14) #based on 7 day MA
predictor.run()


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 14, 50)         │        11,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 14, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,451 (122.86 KB)

 Trainable params: 31,451 (122.86 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0406 - mean_absolute_error: 0.1404 - val_loss: 0.0617 - val_mean_absolute_error: 0.1844
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0153 - mean_absolute_error: 0.1154 - val_loss: 0.0191 - val_mean_absolute_error: 0.1265
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0277 - mean_absolute_error: 0.1600 - val_loss: 0.0111 - val_mean_absolute_error: 0.0955
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0131 - mean_absolute_error: 0.1071 - val_loss: 0.0043 - val_mean_absolute_error: 0.0573
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0042 - mean_absolute_error: 0.0590 - val_loss: 0.0024 - val_mean_absolute_error: 0.0407
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0020 - mean_absolute_error: 0.0400 - val_loss: 0.0018 - val_mean_absolute_error: 0.0331
Epoch 7/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0013 - mean_absolute_error: 0.0305 - v

np.float64(136.52526763400215)

# Adding Reddit Sentiment Analysis

In [14]:
import praw
from textblob import TextBlob
import requests
with open ('pw.txt', 'r') as f:
    pw=f.read()
    
with open ('client_id.txt', 'r') as f:
    client_id=f.read()
    
with open ('client_secret.txt', 'r') as f:
    client_secret=f.read()

auth=requests.auth.HTTPBasicAuth(client_id, client_secret)
# Define your Reddit API credentials
data={"grant_type":"password",
      "username":"InterestingRun2732",
      "password":pw}

headers = {"User-Agent":"MyAPI/0.0.1"}

res=requests.post("https://www.reddit.com/api/v1/access_token", auth=auth, data=data, headers=headers)

res.json()

TOKEN=res.json()['access_token']

In [15]:
headers = {**headers, **{'Authorization':f"bearer {TOKEN}"}} #merge the two dictionaries
headers

requests.get("https://oauth.reddit.com/api/v1/me", headers=headers).json() #check if the token is working

{'is_employee': False,
 'seen_layout_switch': False,
 'has_visited_new_profile': False,
 'pref_no_profanity': True,
 'has_external_account': False,
 'pref_geopopular': '',
 'seen_redesign_modal': False,
 'pref_show_trending': True,
 'subreddit': {'default_set': True,
  'user_is_contributor': False,
  'banner_img': '',
  'restrict_posting': True,
  'user_is_banned': False,
  'free_form_reports': True,
  'community_icon': None,
  'show_media': True,
  'icon_color': '',
  'user_is_muted': None,
  'display_name': 'u_InterestingRun2732',
  'header_img': None,
  'title': '',
  'coins': 0,
  'previous_names': [],
  'over_18': False,
  'icon_size': [256, 256],
  'primary_color': '',
  'icon_img': 'https://styles.redditmedia.com/t5_42x32f/styles/profileIcon_snoo4fb4ed4e-77b2-4f35-a5af-0d69e2dc5dc7-headshot.png?width=256&amp;height=256&amp;crop=256:256,smart&amp;s=7e0320562131de2ac8aab15f31e1abddb358aca6',
  'description': '',
  'allowed_media_in_comments': [],
  'submit_link_label': '',
  'head

In [16]:
import requests

#res = requests.get("https://oauth.reddit.com/r/wallstreetbets/new", headers=headers, params={"limit": 100}) 
subreddits = ['wallstreetbets', 'cryptocurrency', 'stocks']
all_data = {}

for subreddit in subreddits:
    res = requests.get(f"https://oauth.reddit.com/r/{subreddit}/new", headers=headers, params={"limit": 100})
    posts = res.json()['data']['children']
    
    data = []
    for post in posts:
        blob = TextBlob(post['data']['title'])
        data.append({
            'subreddit': post['data']['subreddit'],
            'title': post['data']['title'],
            'selftext': post['data']['selftext'],
            'upvote_ratio': post['data']['upvote_ratio'],
            'upvotes': post['data']['ups'],
            'score': post['data']['score'],
            'polarity': blob.sentiment.polarity
        })
    
    all_data[subreddit] = pd.DataFrame(data)

# Save each subreddit data to a separate CSV file
# Combine all subreddit data into a single DataFrame
combined_df = pd.concat(all_data.values(), ignore_index=True)
#combined_df.to_csv('reddit_data.csv', index=False)
print(combined_df.head())
print('--------')


# Ensure combined_df is defined
if 'combined_df' not in globals():
    raise ValueError("The combined_df DataFrame is not defined. Please ensure it is created in a previous cell.")

# Filter combined_df to only include rows that mention the ticker or similar words
filtered_df = combined_df[combined_df['selftext'].str.contains(ticker, case=False, na=False) | combined_df['title'].str.contains(ticker, case=False, na=False)]

# Print the filtered DataFrame
print(filtered_df.head())
print(filtered_df.columns)



        subreddit                                              title  \
0  wallstreetbets                                            Wipeout   
1  wallstreetbets                           QQQ bet not a smart idea   
2  wallstreetbets                            1k -&gt; 18k Major Gain   
3  wallstreetbets  $130k TLT $90c 2/28 (Fed Dovish or Weakening E...   
4  wallstreetbets                                         $KODK YOLO   

                                            selftext  upvote_ratio  upvotes  \
0  I've been actively buying and selling shares f...          0.86       16   
1  Bought 10 contracts thinking the market would ...          0.67        1   
2  [I am whole AGAIN!!!!](https://preview.redd.it...          0.67        2   
3                                                             1.00        7   
4  https://preview.redd.it/msq060xf8tfe1.png?widt...          1.00        3   

   score  polarity  
0     16  0.000000  
1      1 -0.107143  
2      2  0.062500  
3      7

In [17]:
# Ensure combined_df is defined
if 'combined_df' not in globals():
    raise ValueError("The combined_df DataFrame is not defined. Please ensure it is created in a previous cell.")

# Filter combined_df to only include rows that mention the ticker
filtered_df = combined_df[combined_df['selftext'].str.contains(ticker) | combined_df['title'].str.contains(ticker)]

# Print the filtered DataFrame
print(filtered_df)


          subreddit                                              title  \
9    wallstreetbets    DeepSeek training cost - the $95 mil difference   
10   wallstreetbets                                    NVDA theta play   
15   wallstreetbets                         Perfectly timed NVDA trade   
20   wallstreetbets                                   $NVDA $12k short   
23   wallstreetbets          Not buying the NVDA dip yesterday be like   
34   wallstreetbets  2025 Year-to-Date GAIN [ $258k ] Wheeling TSLA...   
39   wallstreetbets  Grabbed NVDA at 117 for almost 4K of canuck bu...   
40   wallstreetbets           Don’t Worry I’m Playing the Bounce $NVDA   
49   wallstreetbets  12K Into 130 NVDA Calls for MAR 7. Realest DD ...   
53   wallstreetbets                                          AMD YOLO.   
54   wallstreetbets                   Only thing I'm deep will be NVDA   
55   wallstreetbets  Gg boys. I’ll be rich by tomorrow or be homele...   
58   wallstreetbets    $31k NVDA Calls

## Second Model with Reddit Sentiment Analysis

In [18]:
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd
from datetime import datetime

class EnhancedStockPredictor:
    def __init__(self, stock_data, reddit_data, lookback_days=14, train_split=0.7):
        self.stock_data = stock_data
        self.reddit_data = reddit_data
        self.lookback_days = lookback_days
        self.train_split = train_split
        self.price_scaler = MinMaxScaler(feature_range=(0, 1))
        self.sentiment_scaler = MinMaxScaler(feature_range=(0, 1))
        self.model = None
        
    def prepare_data(self):
        # Prepare price features
        price_features = self.stock_data[['Open', 'High', 'Low', 'Close', 'Volume']].values
        normalized_price = self.price_scaler.fit_transform(price_features)
        
        # Convert stock data index to datetime if not already
        self.stock_data['Date'] = pd.to_datetime(self.stock_data['Date'])
        
        # Prepare sentiment features
        self.reddit_data['date'] = pd.to_datetime(self.reddit_data['date'])
        
        # Create a date range covering the stock data period
        date_range = pd.date_range(start=self.stock_data['Date'].min(),
                                    end=self.stock_data['Date'].max(),
                                    freq='D')
        
        # Aggregate sentiment metrics per day
        daily_sentiment = self.reddit_data.groupby('date').agg({
            'polarity': ['mean', 'std'],
            'upvote_ratio': 'mean',
            'score': ['mean', 'sum'],
            'upvotes': 'sum'
        }).fillna(0)
        
        # Flatten column names
        daily_sentiment.columns = ['avg_polarity', 'std_polarity', 'avg_upvote_ratio', 
                                    'avg_score', 'total_score', 'total_upvotes']
        
        # Reindex to include all dates and forward fill, then fill remaining NaNs with 0
        daily_sentiment = daily_sentiment.reindex(date_range)\
                                            .fillna(method='ffill')\
                                            .fillna(0)
        
        # Align sentiment data with stock data dates
        aligned_sentiment = daily_sentiment.loc[self.stock_data['Date']]
        
        # Scale sentiment features
        sentiment_values = aligned_sentiment.values
        normalized_sentiment = self.sentiment_scaler.fit_transform(sentiment_values)
        
        # Combine features
        self.combined_features = np.hstack((normalized_price, normalized_sentiment))
        
        # Create sequences
        X, y = [], []
        for i in range(len(self.combined_features) - self.lookback_days):
            X.append(self.combined_features[i:(i + self.lookback_days)])
            y.append(self.combined_features[i + self.lookback_days, 3])  # Next day's closing price
            
        X, y = np.array(X), np.array(y)
        
        # Split into train and test sets
        train_size = int(len(X) * self.train_split)
        self.X_train, self.X_test = X[:train_size], X[train_size:]
        self.y_train, self.y_test = y[:train_size], y[train_size:]
        
    def build_model(self):
        self.model = tf.keras.Sequential([
            tf.keras.layers.LSTM(64, return_sequences=True, 
                                    input_shape=(self.lookback_days, self.combined_features.shape[1])),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.LSTM(32, return_sequences=True),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.LSTM(16),
            tf.keras.layers.Dense(8, activation='relu'),
            tf.keras.layers.Dense(1)
        ])
        
        self.model.compile(
            optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
            loss='mse',
            metrics=['mae']
        )
        
    def train(self, epochs=100, batch_size=64):
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )
        
        history = self.model.fit(
            self.X_train, self.y_train,
            epochs=epochs,
            batch_size=batch_size,
            validation_split=0.3,
            callbacks=[early_stopping],
            verbose=1
        )
        
        return history

    def evaluate(self):
        predictions = self.model.predict(self.X_test)
        
        # Un-normalize predictions and actual values
        dummy_pred = np.zeros((len(predictions), self.price_scaler.scale_.shape[0]))
        dummy_actual = np.zeros((len(self.y_test), self.price_scaler.scale_.shape[0]))
        
        dummy_pred[:, 3] = predictions.flatten()
        dummy_actual[:, 3] = self.y_test
        
        # Inverse transform
        unnormalized_predictions = self.price_scaler.inverse_transform(dummy_pred)[:, 3]
        unnormalized_actual = self.price_scaler.inverse_transform(dummy_actual)[:, 3]
        
        # Calculate metrics on unnormalized values
        mse = mean_squared_error(unnormalized_actual, unnormalized_predictions)
        mae = mean_absolute_error(unnormalized_actual, unnormalized_predictions)
        rmse = np.sqrt(mse)
        
        print(f'Test MSE: {mse:.4f}')
        print(f'Test MAE: {mae:.4f}')
        print(f'Test RMSE: {rmse:.4f}')
        
        return unnormalized_predictions, unnormalized_actual
        
    def get_dates_for_test(self):
        # Get the dates corresponding to the test set
        train_size = int(len(self.stock_data) * self.train_split)
        # Calculate the correct number of dates based on test data length
        test_size = len(self.y_test)  # Use the length of test data
        test_dates = self.stock_data['Date'].iloc[-(test_size):]  # Get the last test_size dates
        return test_dates

    def predict_next_day(self):
        # Get the last sequence of data
        last_sequence = self.combined_features[-self.lookback_days:]
        last_sequence = last_sequence.reshape((1, self.lookback_days, self.combined_features.shape[1]))
        
        # Make prediction
        predicted_scaled = self.model.predict(last_sequence)
        
        # Create dummy array for inverse transform
        dummy = np.zeros((1, self.price_scaler.scale_.shape[0]))
        dummy[0, 3] = predicted_scaled[0, 0]  # Put predicted value in Close price position
        
        # Inverse transform to get actual price
        predicted_price = self.price_scaler.inverse_transform(dummy)[0, 3]
        
        return predicted_price
    

        

 

# Ensure the filtered_df is defined

In [19]:
# Initialize date for Reddit data
filtered_df['date'] = pd.to_datetime(datetime.now().date())

try:
    # Initialize and train the model
    predictor = EnhancedStockPredictor(data_unsorted, filtered_df)
    predictor.prepare_data()
    predictor.build_model()
    history = predictor.train()

    # Make predictions and get actual values
    predictions, actual_values = predictor.evaluate()
    next_day_price = predictor.predict_next_day()
    test_dates = predictor.get_dates_for_test()

    # Verify dimensions match
    if len(test_dates) != len(predictions):
        min_length = min(len(test_dates), len(predictions))
        test_dates = test_dates[-min_length:]
        predictions = predictions[-min_length:]
        actual_values = actual_values[-min_length:]

    # Create figure with secondary y-axis
    fig = go.Figure()

    # Add candlestick
    fig.add_trace(go.Candlestick(
        x=test_dates,
        open=data_unsorted['Open'].iloc[-len(test_dates):],
        high=data_unsorted['High'].iloc[-len(test_dates):],
        low=data_unsorted['Low'].iloc[-len(test_dates):],
        close=data_unsorted['Close'].iloc[-len(test_dates):],
        name='Actual Price'
    ))

    # Add traces for predicted and actual values
    fig.add_trace(go.Scatter(
        x=test_dates,
        y=predictions,
        mode='lines',
        name='Predicted',
        line=dict(color='red', width=2)
    ))

    # Update layout
    fig.update_layout(
        title=f'{ticker} Stock Price Prediction with Candlestick Chart with Reddit Sentiment Analysis',
        yaxis_title='Stock Price ($)',
        xaxis_title='Date',
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=3, label="3m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(visible=True),
            type="date"
        )
    )

    # Show the plot
    fig.show()

    # Plot the prediction error
    error_fig = go.Figure()
    error = actual_values - predictions
    error_fig.add_trace(go.Scatter(
        x=test_dates,
        y=error,
        mode='lines',
        name='Prediction Error',
        line=dict(color='orange', width=2)
    ))

    error_fig.update_layout(
        title='Prediction Error Over Time',
        yaxis_title='Error ($)',
        xaxis_title='Date',
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=3, label="3m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(visible=True),
            type="date"
        )
    )

    error_fig.show()

    # Calculate and print metrics
    mean_error = np.mean(np.abs(error))
    std_error = np.std(error)
    print(f"\nPredicted next day closing price on {datetime.now().date() + timedelta(days=1)}: ${next_day_price:.2f}")
    print(f"Mean Absolute Error: ${mean_error:.2f}")
    print(f"Standard Deviation of Error: ${std_error:.2f}")
    print(f"Maximum Error: ${np.max(np.abs(error)):.2f}")

except Exception as e:
    print(f"An error occurred: {str(e)}")
    import traceback
    traceback.print_exc()

Epoch 1/100


/var/folders/53/kssq81zs06d17_l658v4lbnm0000gn/T/ipykernel_54492/2678640983.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/53/kssq81zs06d17_l658v4lbnm0000gn/T/ipykernel_54492/3096575698.py:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/53/kssq81zs06d17_l658v4lbnm0000gn/T/ipykernel_54492/3096575698.py:48: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/Library/Frameworks/Python.framewo

6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 59ms/step - loss: 0.0080 - mae: 0.0657 - val_loss: 0.0330 - val_mae: 0.1660
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0030 - mae: 0.0460 - val_loss: 0.0203 - val_mae: 0.1262
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0024 - mae: 0.0412 - val_loss: 0.0215 - val_mae: 0.1326
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0017 - mae: 0.0315 - val_loss: 0.0160 - val_mae: 0.1137
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0012 - mae: 0.0263 - val_loss: 0.0059 - val_mae: 0.0624
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 5.7401e-04 - mae: 0.0194 - val_loss: 0.0014 - val_mae: 0.0286
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 3.9708e-04 - mae: 0.0152 - val_loss: 0.0010 - val_mae: 0.0264
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 3.0602e-04 - mae: 0.0139 - val_loss: 0.0012 - val_mae: 0.0283
Epoch 9/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 3.5471e-04 - m


Predicted next day closing price on 2025-01-29: $105.12
Mean Absolute Error: $21.53
Standard Deviation of Error: $13.63
Maximum Error: $45.48


## Incorporate Alpha Vantage and Yahoo News Sentiment Analysis

In [20]:
import requests
import pandas as pd
from textblob import TextBlob
from datetime import datetime, timedelta
import yfinance as yf
from bs4 import BeautifulSoup

class NewsCollector:
    def __init__(self, alpha_vantage_key):
        """Initialize with Alpha Vantage API key (free)"""
        self.alpha_vantage_key = alpha_vantage_key
        
    def get_alpha_vantage_news(self, ticker, limit=50):
        """Get news from Alpha Vantage API"""
        base_url = "https://www.alphavantage.co/query"
        params = {
            "function": "NEWS_SENTIMENT",
            "tickers": ticker,
            "apikey": self.alpha_vantage_key,
            "limit": limit
        }
        
        try:
            response = requests.get(base_url, params=params)
            data = response.json()
            
            if "feed" not in data:
                print("No news data found or API limit reached")
                return pd.DataFrame()
            
            news_data = []
            for article in data["feed"]:
                # Get sentiment from TextBlob as backup
                blob = TextBlob(article["title"] + " " + article["summary"])
                textblob_sentiment = blob.sentiment.polarity
                
                news_data.append({
                    'date': article["time_published"],
                    'title': article["title"],
                    'summary': article["summary"],
                    'source': article["source"],
                    'url': article["url"],
                    'sentiment': article.get("overall_sentiment_score", textblob_sentiment),
                    'sentiment_label': article.get("overall_sentiment_label", "neutral")
                })
                
            return pd.DataFrame(news_data)
        
        except Exception as e:
            print(f"Error collecting Alpha Vantage news: {str(e)}")
            return pd.DataFrame()
    
    def get_yahoo_news(self, ticker, days=30):
        """Get news from Yahoo Finance"""
        try:
            stock = yf.Ticker(ticker)
            news = stock.news
            
            if not news:
                return pd.DataFrame()
            
            news_data = []
            for article in news:
                try:
                    # More robust data extraction with error handling
                    title = article.get('title', '')
                    if not title:  # Skip articles without titles
                        continue
                        
                    # Get sentiment
                    blob = TextBlob(title)
                    sentiment = blob.sentiment.polarity
                    
                    # Extract other fields with fallbacks
                    news_data.append({
                        'date': datetime.fromtimestamp(article.get('providerPublishTime', datetime.now().timestamp())),
                        'title': title,
                        'source': article.get('publisher', 'Unknown'),
                        'url': article.get('link', ''),
                        'polarity': sentiment
                    })
                except Exception as article_error:
                    print(f"Error processing article: {str(article_error)}")
                    continue
            
            return pd.DataFrame(news_data)
            
        except Exception as e:
            print(f"Error collecting Yahoo news: {str(e)}")
            return pd.DataFrame()
            
    
    def get_combined_sentiment(self, ticker, days=30):
        """Combine news from multiple sources and get sentiment"""
        try:
            # Get news from both sources
            alpha_news = self.get_alpha_vantage_news(ticker)
            yahoo_news = self.get_yahoo_news(ticker)
            
            # Convert dates to datetime
            if not alpha_news.empty:
                alpha_news['date'] = pd.to_datetime(alpha_news['date'])
            if not yahoo_news.empty:
                yahoo_news['date'] = pd.to_datetime(yahoo_news['date'])
            
            # Filter for last n days
            start_date = datetime.now() - timedelta(days=days)
            
            if not alpha_news.empty:
                alpha_news = alpha_news[alpha_news['date'] >= start_date]
            if not yahoo_news.empty:
                yahoo_news = yahoo_news[yahoo_news['date'] >= start_date]
            
            # Print debug info
            print(f"Alpha Vantage news count: {len(alpha_news)}")
            print(f"Yahoo Finance news count: {len(yahoo_news)}")
            
            # Combine and aggregate daily sentiment
            all_news = pd.concat([
                alpha_news[['date', 'sentiment']].rename(columns={'sentiment': 'polarity'}) if not alpha_news.empty else pd.DataFrame(),
                yahoo_news[['date', 'polarity']] if not yahoo_news.empty else pd.DataFrame()
            ])
            
            if all_news.empty:
                print("No news data available")
                return pd.DataFrame()
            
            # Aggregate by date
            daily_sentiment = all_news.groupby(all_news['date'].dt.date).agg({
                'polarity': ['mean', 'std', 'count']
            }).reset_index()
            
            # Flatten column names
            daily_sentiment.columns = ['date', 'avg_polarity', 'std_polarity', 'article_count']
            
            return daily_sentiment
            
        except Exception as e:
            print(f"Error in get_combined_sentiment: {str(e)}")
            return pd.DataFrame()


# Get your Alpha Vantage API key from: https://www.alphavantage.co/support/#api-key
with open ('alphavantage.txt', 'r') as f:
    alpha_vantage_key=f.read()
    
# Initialize collector
collector = NewsCollector(alpha_vantage_key)
sentiment_data = collector.get_combined_sentiment(ticker=ticker)
print(sentiment_data)

alpha_news = collector.get_alpha_vantage_news(ticker=ticker)
print("\nRecent news headlines:")
for _, row in alpha_news.iterrows():
    print(f"{row['date']}: {row['title']} (Sentiment: {row['sentiment_label']})")


No news data found or API limit reached
Alpha Vantage news count: 0
Yahoo Finance news count: 0
No news data available
Empty DataFrame
Columns: []
Index: []
No news data found or API limit reached

Recent news headlines:


## Third Model with Alpha Vantage and Yahoo News

In [21]:
class FinalPredictor:
    def __init__(self, stock_data, reddit_data, news_collector, lookback_days=60, train_split=0.7):
        self.stock_data = stock_data
        self.reddit_data = reddit_data
        self.news_collector = news_collector
        self.lookback_days = lookback_days
        self.train_split = train_split
        self.price_scaler = MinMaxScaler(feature_range=(0, 1))
        self.sentiment_scaler = MinMaxScaler(feature_range=(0, 1))
        self.model = None
        
    def prepare_data(self):
        # 1. Prepare stock price features
        price_features = self.stock_data[['Open', 'High', 'Low', 'Close', 'Volume']].values
        normalized_price = self.price_scaler.fit_transform(price_features)
        
        # 2. Prepare Reddit sentiment features
        self.reddit_data['date'] = pd.to_datetime(self.reddit_data['date'])
        reddit_sentiment = self.reddit_data.groupby('date').agg({
            'polarity': ['mean', 'std'],
            'upvote_ratio': 'mean',
            'score': ['mean', 'sum']
        }).fillna(0)
        
        # Flatten Reddit column names
        reddit_sentiment.columns = ['reddit_avg_polarity', 'reddit_std_polarity', 
                                  'reddit_avg_upvote', 'reddit_avg_score', 'reddit_total_score']
        
        # 3. Prepare News sentiment features
        news_sentiment = self.news_collector.get_combined_sentiment(ticker=ticker)
        if not news_sentiment.empty:
            news_sentiment.set_index('date', inplace=True)
            news_sentiment.columns = ['news_avg_polarity', 'news_std_polarity', 'news_article_count']
        
        # 4. Align all dates
        date_range = pd.date_range(start=self.stock_data['Date'].min(),
                                   end=self.stock_data['Date'].max(),
                                   freq='D')
        
        # Reindex and forward fill stock data
        stock_data_reindexed = self.stock_data.set_index('Date').reindex(date_range).fillna(method='ffill').fillna(0)
        normalized_price = self.price_scaler.fit_transform(stock_data_reindexed[['Open', 'High', 'Low', 'Close', 'Volume']].values)
        
        # Reindex and forward fill sentiment data
        reddit_sentiment = reddit_sentiment.reindex(date_range).fillna(method='ffill').fillna(0)
        if not news_sentiment.empty:
            news_sentiment = news_sentiment.reindex(date_range).fillna(method='ffill').fillna(0)
        
        # 5. Combine all features
        sentiment_features = pd.concat([reddit_sentiment, news_sentiment], axis=1).fillna(0)
        normalized_sentiment = self.sentiment_scaler.fit_transform(sentiment_features.values)
        
        # 6. Combine price and sentiment features
        self.combined_features = np.hstack((normalized_price, normalized_sentiment))
        
        # 7. Create sequences
        X, y = [], []
        for i in range(len(self.combined_features) - self.lookback_days):
            X.append(self.combined_features[i:(i + self.lookback_days)])
            y.append(self.combined_features[i + self.lookback_days, 3])  # Next day's closing price
            
        X, y = np.array(X), np.array(y)
        
        # 8. Split into train and test sets
        train_size = int(len(X) * self.train_split)
        self.X_train, self.X_test = X[:train_size], X[train_size:]
        self.y_train, self.y_test = y[:train_size], y[train_size:]
        
    def build_model(self):
        input_shape = (self.lookback_days, self.combined_features.shape[1])
        
        # Create a more complex model with attention mechanism
        inputs = tf.keras.Input(shape=input_shape)
        
        # LSTM layers with residual connections
        x = tf.keras.layers.LSTM(128, return_sequences=True)(inputs)
        x = tf.keras.layers.Dropout(0.2)(x)
        
        # Add attention mechanism
        attention = tf.keras.layers.Dense(1)(x)
        attention = tf.keras.layers.Softmax(axis=1)(attention)
        x = tf.keras.layers.Multiply()([x, attention])
        
        # Additional LSTM layers
        x = tf.keras.layers.LSTM(64, return_sequences=True)(x)
        x = tf.keras.layers.Dropout(0.2)(x)
        x = tf.keras.layers.LSTM(32)(x)
        
        # Dense layers with batch normalization
        x = tf.keras.layers.Dense(32, activation='relu')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dense(16, activation='relu')(x)
        outputs = tf.keras.layers.Dense(1)(x)
        
        self.model = tf.keras.Model(inputs=inputs, outputs=outputs)
        
        # Simpler optimizer configuration with fixed learning rate
        self.model.compile(
            optimizer=tf.keras.optimizers.Adamax(learning_rate=0.0001),
            loss='mse',
            metrics=['mae']
        )
        
    def train(self, epochs=200, batch_size=64):
        # Add callbacks
        #epochs=300
        callbacks = [
            tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=15,
                restore_best_weights=True
            ),
            tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=10,
                min_lr=0.00001
            )
        ]
        
        history = self.model.fit(
            self.X_train, self.y_train,
            epochs=epochs,
            batch_size=batch_size,
            validation_split=0.3,
            callbacks=callbacks,
            verbose=1
        )
        
        return history
    
    def predict_next_day(self):
        # Get the last sequence of data
        last_sequence = self.combined_features[-self.lookback_days:]
        last_sequence = last_sequence.reshape((1, self.lookback_days, self.combined_features.shape[1]))
        
        # Make prediction
        predicted_scaled = self.model.predict(last_sequence)
        
        # Create dummy array for inverse transform
        dummy = np.zeros((1, self.price_scaler.scale_.shape[0]))
        dummy[0, 3] = predicted_scaled[0, 0]
        
        # Inverse transform to get actual price
        predicted_price = self.price_scaler.inverse_transform(dummy)[0, 3]
        
        return predicted_price
    
    def evaluate(self):
        predictions = self.model.predict(self.X_test)
        
        # Un-normalize predictions and actual values
        dummy_pred = np.zeros((len(predictions), self.price_scaler.scale_.shape[0]))
        dummy_actual = np.zeros((len(self.y_test), self.price_scaler.scale_.shape[0]))
        
        dummy_pred[:, 3] = predictions.flatten()
        dummy_actual[:, 3] = self.y_test
        
        # Inverse transform
        unnormalized_predictions = self.price_scaler.inverse_transform(dummy_pred)[:, 3]
        unnormalized_actual = self.price_scaler.inverse_transform(dummy_actual)[:, 3]
        
        # Calculate metrics
        mse = mean_squared_error(unnormalized_actual, unnormalized_predictions)
        mae = mean_absolute_error(unnormalized_actual, unnormalized_predictions)
        rmse = np.sqrt(mse)
        
        print(f'Test MSE: ${mse:.2f}')
        print(f'Test MAE: ${mae:.2f}')
        print(f'Test RMSE: ${rmse:.2f}')
        
        return unnormalized_predictions, unnormalized_actual
    
    def get_dates_for_test(self):
        """Get the dates corresponding to the test set"""
        train_size = int(len(self.stock_data) * self.train_split)
        test_size = len(self.y_test)
        test_dates = self.stock_data['Date'].iloc[-(test_size):]
        return test_dates




In [22]:
# Initialize and train the model
final_predictor = FinalPredictor(
    stock_data=data_unsorted,
    reddit_data=filtered_df,
    news_collector=collector,
    lookback_days=14
)

try:
    # Prepare data and train model
    final_predictor.prepare_data()
    final_predictor.build_model()
    history = final_predictor.train(epochs=300)

    # Make predictions
    predictions, actual_values = final_predictor.evaluate()
    next_day_price = final_predictor.predict_next_day()
    test_dates = final_predictor.get_dates_for_test()

    # Verify dimensions match
    if len(test_dates) != len(predictions):
        min_length = min(len(test_dates), len(predictions))
        test_dates = test_dates[-min_length:]
        predictions = predictions[-min_length:]
        actual_values = actual_values[-min_length:]

    # Create figure with candlestick
    fig = go.Figure()

    # Add candlestick
    fig.add_trace(go.Candlestick(
        x=test_dates,
        open=data_unsorted['Open'].iloc[-len(test_dates):],
        high=data_unsorted['High'].iloc[-len(test_dates):],
        low=data_unsorted['Low'].iloc[-len(test_dates):],
        close=data_unsorted['Close'].iloc[-len(test_dates):],
        name='Actual Price'
    ))

    # Add prediction line
    fig.add_trace(go.Scatter(
        x=test_dates,
        y=predictions,
        mode='lines',
        name='Predicted',
        line=dict(color='red', width=2)
    ))

    # Update layout
    fig.update_layout(
        title=f'{ticker} Stock Price Prediction with News and Reddit Sentiment Analysis',
        yaxis_title='Stock Price ($)',
        xaxis_title='Date',
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=3, label="3m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(visible=True),
            type="date"
        )
    )

    # Show the main plot
    fig.show()

    # Create error plot
    error_fig = go.Figure()
    error = actual_values - predictions
    
    error_fig.add_trace(go.Scatter(
        x=test_dates,
        y=error,
        mode='lines',
        name='Prediction Error',
        line=dict(color='orange', width=2)
    ))

    error_fig.update_layout(
        title='Prediction Error Over Time',
        yaxis_title='Error ($)',
        xaxis_title='Date',
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=3, label="3m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(visible=True),
            type="date"
        )
    )

    # Show the error plot
    error_fig.show()

    # Print metrics
    print(f"\nPredicted next day closing price: ${next_day_price:.2f}")
    print(f"Mean Absolute Error: ${np.mean(np.abs(error)):.2f}")
    print(f"Standard Deviation of Error: ${np.std(error):.2f}")
    print(f"Maximum Error: ${np.max(np.abs(error)):.2f}")

except Exception as e:
    print(f"An error occurred: {str(e)}")
    import traceback
    traceback.print_exc()

/var/folders/53/kssq81zs06d17_l658v4lbnm0000gn/T/ipykernel_54492/2944787770.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



No news data found or API limit reached
Alpha Vantage news count: 0
Yahoo Finance news count: 0
No news data available
Epoch 1/300


/var/folders/53/kssq81zs06d17_l658v4lbnm0000gn/T/ipykernel_54492/2944787770.py:41: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/var/folders/53/kssq81zs06d17_l658v4lbnm0000gn/T/ipykernel_54492/2944787770.py:45: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - loss: 0.0100 - mae: 0.0788 - val_loss: 0.0919 - val_mae: 0.2908 - learning_rate: 1.0000e-04
Epoch 2/300
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0083 - mae: 0.0736 - val_loss: 0.0891 - val_mae: 0.2861 - learning_rate: 1.0000e-04
Epoch 3/300
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.0068 - mae: 0.0660 - val_loss: 0.0866 - val_mae: 0.2817 - learning_rate: 1.0000e-04
Epoch 4/300
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0054 - mae: 0.0606 - val_loss: 0.0840 - val_mae: 0.2770 - learning_rate: 1.0000e-04
Epoch 5/300
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.0044 - mae: 0.0537 - val_loss: 0.0815 - val_mae: 0.2725 - learning_rate: 1.0000e-04
Epoch 6/300
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0032 - mae: 0.0471 - val_loss: 0.0793 - val_mae: 0.2683 - learning_rate: 1.0000e-04
Epoch 7/300
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.0029 - mae: 0.0448 - val_loss: 0.0771 - val_mae: 0.2643 - learning_rate: 1.0000e-04
Epoc


Predicted next day closing price: $58.59
Mean Absolute Error: $62.56
Standard Deviation of Error: $17.89
Maximum Error: $90.72
